In [1]:
#All the imports & pyplot inline setting
import numpy as np
import pandas as pd
import random
from sklearn import preprocessing
import scipy.stats

In [16]:
n_file='project3_dataset2.txt'
#n_file='project3_dataset1.txt'
n_folds = 10
n_trees = 5

In [17]:
df = pd.read_csv(n_file,header=None,delim_whitespace=True)

In [18]:
df = pd.read_csv(n_file,header=None,delim_whitespace=True)
shuffled_idx = np.random.permutation(df.index)
df = df.reindex(shuffled_idx)
data_truth = df.iloc[:,-1].values

In [19]:
def create_branches_continous(train,column_index,column_value):
    left = train[train[column_index]<=column_value]
    right = train[train[column_index]>column_value]
    return (left,right)

In [20]:
def create_branches_categorical(train,column_index,column_value):
    left = train[train[column_index]==column_value]
    right = train[train[column_index]!=column_value]
    return (left,right)

In [21]:
def calculate_gini(temp_truth):
    count0 = ((temp_truth==0).sum())
    count1 = ((temp_truth==1).sum())
    total = count0 + count1
    p_count0 = (0.0 + count0)/total
    p_count1 = (0.0 + count1)/total
    gini = 1.0 - p_count0*p_count0 - p_count1*p_count1
    return gini

In [22]:
class TreeNode:
    def __init__(self, column_number, operator, isLeaf, truth, value, left, right):
        self.column_number = column_number
        self.operator = operator
        self.isLeaf = isLeaf
        self.truth = truth
        self.value = value
        self.left = left
        self.right = right
   
    def setLeaf(self, isLeaf, truth):
        self.isLeaf = isLeaf
        self.truth = truth

    def setChild(self,column_number, operator, value, left, right):
        self.column_number = column_number
        self.operator = operator
        self.value = value
        self.left = left
        self.right = right

In [23]:
def build_tree(node_tree, node):
    global n_count
    column_count = node.shape[1] - 1
    record_count = node.shape[0]
    node_truth = node.iloc[:,-1].values
    node_truth = node_truth.reshape((node_truth.shape[0],1))
    current_gini = calculate_gini(node_truth)
        
    column_number = None
    operator = None
    isLeaf = False
    truth = None
    value = None
    gini_combined = 0.0
    min_gini = current_gini
    
    for i in range(column_count):
        if(node[i].dtype=='object'):
            le = preprocessing.LabelEncoder()
            le.fit(node[:][i])
            l_node = le.transform(node[:][i])
            l_node = l_node.reshape((l_node.shape[0], 1))
            for j in range(len(le.classes_)):
                truth_left = node_truth[l_node==j]
                truth_right = node_truth[l_node!=j]
                if(truth_left.shape[0]==0 or truth_right.shape[0]==0):
                    continue
                gini_child_left = calculate_gini(truth_left)
                gini_child_right = calculate_gini(truth_right)
                gini_combined = (0.0 + truth_left.shape[0])/(0.0 + record_count)*gini_child_left \
                + (0.0 + truth_right.shape[0])/(0.0 + record_count)*gini_child_right
            
                if gini_combined < min_gini:
                    column_number = i
                    operator = '=='
                    value = le.classes_[j]
                    isLeaf = False
                    min_gini = gini_combined
        else:
            col = node[i]
            col_mean = col.mean()
            truth_left = node_truth[(col<=col_mean).reshape((col.shape[0],1))]
            truth_right = node_truth[(col>col_mean).reshape((col.shape[0],1))]
            if(truth_left.shape[0]==0 or truth_right.shape[0]==0):
                    continue
            gini_child_left = calculate_gini(truth_left)
            gini_child_right = calculate_gini(truth_right)
            gini_combined = (0.0 + truth_left.shape[0])/(0.0 + record_count)*gini_child_left \
            + (0.0 + truth_right.shape[0])/(0.0 + record_count)*gini_child_right
            
            if gini_combined < min_gini:
                column_number = i
                operator = '<='
                value = col_mean
                isLeaf = False
                min_gini = gini_combined

    if min_gini == 0.0 or column_number is None:
        isLeaf = True
        truth_0 = node_truth[node_truth==0]
        truth_1 = node_truth[node_truth==1]
        if truth_0.shape[0] >= truth_1.shape[0]:
            truth = 0
        else:
            truth = 1
        node_tree.setLeaf(isLeaf, truth)
    else:
        if(operator=='=='):
            left,right = create_branches_categorical(node,column_number,value)
        else:
            left,right = create_branches_continous(node,column_number,value)
        
        if(left.shape[0]!=0):
            left_child = TreeNode(None, None, False, None, None, None, None)
            build_tree(left_child,left)
        else:
            left_child = None
        
        if(right.shape[0]!=0):
            right_child = TreeNode(None, None, False, None, None, None, None)
            build_tree(right_child,right)
        else:
            right_child = None
        
        if(left_child is not None and right_child is not None and left_child.isLeaf and right_child.isLeaf and left_child.truth==right_child.truth):
            node_tree.setLeaf(True,left_child.truth)
            node_tree.setChild(None, None, None, None, None)
        else:
            node_tree.setChild(column_number, operator, value, left_child, right_child)
    

In [24]:
def printtree(tree,indent=''):
    # Is this a leaf node?
    if tree.isLeaf:
        print str(tree.truth)
    else:
        # Print the criteria
        print str(tree.column_number)+tree.operator+str(tree.value)+'? '

        # Print the branches
        print indent+'T->',
        printtree(tree.left,indent+'|  ')
        print indent+'F->',
        printtree(tree.right,indent+'  ')

In [25]:
#printtree(root_node)

In [26]:
def find_label(test_rec,tree):
    while not tree.isLeaf:
        if tree.operator=='==':
            if test_rec[tree.column_number]==tree.value:
                tree = tree.left
            else:
                tree = tree.right
        else:
            if test_rec[tree.column_number]<=tree.value:
                tree = tree.left
            else:
                tree = tree.right
    return tree.truth

In [27]:
n_features = df.shape[1]//5.0

In [28]:
def calculate_measure(data_label,test_truth):
    total = data_label.shape[0]
    match = data_label[data_label.flatten()==test_truth.flatten()]
    non_match = data_label[data_label.flatten()!=test_truth.flatten()]
    pos = match[match==1].shape[0]
    neg = match[match==0].shape[0]
    false_pos = non_match[non_match==1].shape[0]
    false_neg = non_match[non_match==0].shape[0]
    cal_accuracy = (0.0 + pos + neg)/(0.0 + pos + neg + false_pos + false_neg)
    cal_precision = (0.0 + pos)/(0.0 + pos + neg)
    cal_recall = (0.0 + pos)/(0.0 + pos + false_neg)
    denom = (0.0 + cal_precision + cal_recall)
    if denom==0:
        denom=1
    cal_f_measure = (2.0*cal_precision*cal_recall)/denom
    return (cal_accuracy,cal_precision,cal_recall,cal_f_measure)

In [29]:
accuracy = 0.0
precision = 0.0
recall = 0.0
f_measure = 0.0
count=shuffled_idx.shape[0]//n_folds
for i in range(n_folds):
    test = df.iloc[count*i:count*(i+1)]
    train = pd.concat([df.iloc[0:count*i],df.iloc[count*(i+1):]])
    test_truth = data_truth[count*i:count*(i+1)]
    train_truth = pd.concat([df.iloc[:,-1].iloc[0:count*i],df.iloc[:,-1].iloc[count*(i+1):]])
    
    prob = np.ones(train.shape[0])
    prob = prob/prob.sum()
    train2 = train
    train_truth2 = train_truth
    train2.index = np.arange(train.shape[0])
    train_truth2.index = np.arange(train.shape[0])
    original_index = np.arange(train.shape[0])
    root_node_list = []
    accuracy_boosting = np.zeros(n_trees)
    for k in range(n_trees):
        
        replacement_idx = np.random.choice(train.shape[0], train.shape[0]//10*9, replace=False, p=prob)
        validation_idx = np.setdiff1d(original_index, replacement_idx)
        
        train3 = train2.reindex(replacement_idx)
        validation3 = train2.reindex(validation_idx)
        train_truth2 = train_truth.reindex(replacement_idx).values
        validation_truth3 = train_truth.reindex(validation_idx).values
        data_label = np.zeros(validation_truth3.shape[0])
            
        root_node_list.append(TreeNode(None, None, False, None, None, None, None))
        build_tree(root_node_list[k], train3)
        for j in range(validation_truth3.shape[0]):
            data_label[j] = find_label(validation3.iloc[j],root_node_list[k])
            if(data_label[j]!=validation_truth3[j]):
                prob[validation_idx[j]] = prob[validation_idx[j]] * 2 
        prob = prob/prob.sum()
        
        accuracy_boosting[k] = (0.0 + (data_label.flatten()==validation_truth3.flatten()).sum())/data_label.shape[0]
    
    data_label = np.zeros(test_truth.shape[0])
    for k in range(n_trees):
        for j in range(test_truth.shape[0]):
            data_label[j] = data_label[j] + accuracy_boosting[k]/accuracy_boosting.sum() * find_label(test.iloc[j],root_node_list[k])
    
    data_label[data_label<0.5] = 0
    data_label[data_label>=0.5] = 1
    current_accuracy,current_precision,current_recall,current_f_measure = calculate_measure(data_label,test_truth)
    print 'kFold ' + str(i) + ':\nAccuracy: ' + str(current_accuracy) + '\tPrecision: ' + str(current_precision) + \
    '\tRecall: ' + str(current_recall) + '\tF-measure: ' + str(current_f_measure)
    accuracy = accuracy + current_accuracy
    precision = precision + current_precision
    recall = recall + current_recall
    f_measure = f_measure + current_f_measure
print 'Average Accuracy: ' + str(accuracy/n_folds)
print 'Average Precision: ' + str(precision/n_folds)
print 'Average Recall: ' + str(recall/n_folds)
print 'Average F-Measure: ' + str(f_measure/n_folds)

kFold 0:
Accuracy: 0.652173913043	Precision: 0.166666666667	Recall: 0.384615384615	F-measure: 0.232558139535
kFold 1:
Accuracy: 0.565217391304	Precision: 0.192307692308	Recall: 0.294117647059	F-measure: 0.232558139535
kFold 2:
Accuracy: 0.652173913043	Precision: 0.133333333333	Recall: 0.333333333333	F-measure: 0.190476190476
kFold 3:
Accuracy: 0.652173913043	Precision: 0.166666666667	Recall: 0.3125	F-measure: 0.217391304348
kFold 4:
Accuracy: 0.652173913043	Precision: 0.233333333333	Recall: 0.466666666667	F-measure: 0.311111111111
kFold 5:
Accuracy: 0.652173913043	Precision: 0.233333333333	Recall: 0.466666666667	F-measure: 0.311111111111
kFold 6:
Accuracy: 0.652173913043	Precision: 0.1	Recall: 0.1875	F-measure: 0.130434782609
kFold 7:
Accuracy: 0.652173913043	Precision: 0.366666666667	Recall: 0.55	F-measure: 0.44
kFold 8:
Accuracy: 0.586956521739	Precision: 0.185185185185	Recall: 0.238095238095	F-measure: 0.208333333333
kFold 9:
Accuracy: 0.673913043478	Precision: 0.129032258065	Recall